### Problem 1] Simple Forward propagation implementation of RNN

In [1]:
import numpy as np

class ScratchSimpleRNNClassifier:
    def __init__(self, n_features, n_nodes):
        self.n_features = n_features
        self.n_nodes = n_nodes
        self.Wx = None
        self.Wh = None
        self.b = None

    def initialize_weights(self):
        self.Wx = np.random.randn(self.n_features, self.n_nodes)
        self.Wh = np.random.randn(self.n_nodes, self.n_nodes)
        self.b = np.random.randn(self.n_nodes)

    def forward(self, x):
        batch_size, n_sequences, n_features = x.shape
        h = np.zeros((batch_size, self.n_nodes))
        for t in range(n_sequences):
            a_t = np.dot(x[:, t, :], self.Wx) + np.dot(h, self.Wh) + self.b
            h = np.tanh(a_t)
        return h

# using the implementation
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4

rnn = ScratchSimpleRNNClassifier(n_features=n_features, n_nodes=n_nodes)
rnn.Wx = w_x
rnn.Wh = w_h
rnn.b = b

h = rnn.forward(x)
print(h) 


[[0.79494228 0.81839002 0.83939649 0.85584174]]


### [Problem 2] Experiment of forward propagation with small sequence

In [2]:
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)

In [3]:
print(h)

[[0.79494228 0.81839002 0.83939649 0.85584174]]


### [Problem 3] (Advance assignment) Implementation of backpropagation

In [9]:
class ScratchSimpleRNNClassifier:
    def __init__(self, n_features, n_nodes, learning_rate=0.01):
        self.n_features = n_features
        self.n_nodes = n_nodes
        self.learning_rate = learning_rate
        self.Wx = None
        self.Wh = None
        self.b = None

    def initialize_weights(self):
        self.Wx = np.random.randn(self.n_features, self.n_nodes)
        self.Wh = np.random.randn(self.n_nodes, self.n_nodes)
        self.b = np.random.randn(self.n_nodes)

    def forward(self, x):
        batch_size, n_sequences, n_features = x.shape
        h = np.zeros((batch_size, self.n_nodes))
        self.hs = [h]
        for t in range(n_sequences):
            a_t = np.dot(x[:, t, :], self.Wx) + np.dot(h, self.Wh) + self.b
            h = np.tanh(a_t)
            self.hs.append(h)
        return h

    def backward(self, x, y, h):
        batch_size, n_sequences, n_features = x.shape
        delta_wx = np.zeros_like(self.Wx, dtype=np.float64)
        delta_wh = np.zeros_like(self.Wh, dtype=np.float64)
        delta_b = np.zeros_like(self.b, dtype=np.float64)
        dh_next = np.zeros_like(h, dtype=np.float64)

        for t in reversed(range(n_sequences)):
            dh = (h - y) * (1 - h ** 2) + dh_next
            delta_b += dh.sum(axis=0)
            delta_wx += np.dot(x[:, t, :].T, dh)
            delta_wh += np.dot(self.hs[t].T, dh)
            dh_next = np.dot(dh, self.Wh.T)

        self.Wx -= self.learning_rate * delta_wx / batch_size
        self.Wh -= self.learning_rate * delta_wh / batch_size
        self.b -= self.learning_rate * delta_b / batch_size

In [11]:
# Initialize and test
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
b = np.array([1, 1, 1, 1], dtype=np.float64) # (n_nodes,)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4

rnn = ScratchSimpleRNNClassifier(n_features=n_features, n_nodes=n_nodes)
rnn.Wx = w_x
rnn.Wh = w_h
rnn.b = b

In [12]:
# Forward pass
h = rnn.forward(x)


In [13]:
# Dummy target
y = np.array([[1, 0, 0, 0]])

In [14]:
# Backward pass
rnn.backward(x, y, h)
print(h)

[[0.79494228 0.81839002 0.83939649 0.85584174]]
